# Modeling the Inventory Problem

The basic idea of the inventory problem is to satify a demand of beams in the most cost-effective way possible. Given a demand of different size beams, which sizes are the cheapest to keep in your inventroy?

In [ ]:
from tkinter import *
import numpy as np
from random import randint
from functools import partial
import math

Lets start with a simple demand of a 3, 4, and 5 foot beams. Enumerate each possibility below to find the most cost-effective inventory for the given demand.

In [ ]:
# GUI
master = Tk()
master.geometry("1600x850")
master.title("The Inventory Problem - Toy Example")

# Canvas
canvas = Canvas(master, width=1200, height=750, background='white')
canvas.grid(row=0, column=1, rowspan=15, columnspan=3)
canvas.create_line(0, 690, 1200, 690)
canvas.create_line(0, 440, 1200, 440)
canvas.create_line(0, 240, 1200, 240)


# Functions
def draw_beams(counts, heightAdjust):
    canvas.delete("height450")
    for count in range(3):
        for i in range(counts[count]):
            canvas.create_rectangle(400*(count + 1) + i*50 - 320, 220 - 70*(count + 1) + heightAdjust, 400*(count + 1) +i*50 - 350, 220 + heightAdjust, outline="#fb0", fill="#fb0", tags="height"+(str)(heightAdjust))

def select_arcs(arc_list):
    canvas.delete("arc")
    global costs
    costs=[None,0,0,0]
    for arc in arc_list:
        costs[arc[1]] = arcs[arc[0], arc[1]]
        if arc[1] - arc[0] == 1:
            canvas.create_line(320*arc[0]+55, 280, 320*arc[1]+15,280, fill="red", width=5, tags="arc")
        elif arc[1] - arc[0] == 2:
            canvas.create_arc(320*arc[0] + 35, 230, 320*arc[1] +35, 370, start=180, extent=180, style="arc", outline="red",width=5, tags="arc")
        else:
            canvas.create_arc(320*arc[0] + 35, 170, 320*arc[1] +35, 430, start=180, extent=180, style="arc", outline="red",width=5,tags="arc")
    
def select_node(size):
    canvas.create_oval(320*(size+1) +15, 260, 320*(size+1)+55,300, fill='yellow', tags=("selected","a" + (str)(size)))
    canvas.create_text(320*(size+1) +35, 280, text=(str)(size+3), tags=("selected","a" +(str)(size)))

def draw_costs():
    canvas.delete("cost")
    for i in range(1,4):
        canvas.create_text(400*i -285, 720, text="$" + (str)(costs[i]),font="Ariel 20",tags="cost")
    canvas.create_text(1100, 720, text="$" + (str)(costs[1] + costs[2] + costs[3]), font="Ariel 20",tags="cost")
    
def switch(size):
    if inUse[size]:
        inUse[size] = False
        canvas.delete("a"+(str)(size))
    else:
        inUse[size] = True
        select_node(size)
    update_inv()
    
def update_inv():
    global inventory
    if inUse[0] and inUse[1]:
        inventory = [3,1,2]
        select_arcs([[0,1],[1,2],[2,3]])
    elif inUse[0]:
        inventory = [3,0,3]
        select_arcs([[0,1],[1,3]])
    elif inUse[1]:
        inventory = [0,4,2]
        select_arcs([[0,2],[2,3]])
    else:
        inventory = [0,0,6]
        select_arcs([[0,3]])
    draw_costs()
    draw_beams(inventory, 450)

# Side Labels
inputLabel = Label(master, text= "Inputs: ", font = "Ariel 20", justify = "left")
inputLabel.grid(row=0, column=0)

graphLabel = Label(master, text= "Graph: ", font = "Ariel 20", justify = "left")
graphLabel.grid(row=5, column=0)

outputLabel = Label(master, text= "Feasible Solution: ", font = "Ariel 20", justify = "left")
outputLabel.grid(row=10, column=0)

switches = [None] * 3
for i in range(2):
    switches[i] = Button(master, text="Stock / Unstock Length " + (str)(i+3) +" Beams", command=partial(switch,i))
    switches[i].grid(row=i+11, column=0)
switches[2] = Label(master, text= "Cannot Unstock Length 5 Beams")
switches[2].grid(row=13, column=0)

costLabel = Label(master, text= "Total Cost: ", font = "Ariel 20", justify = "left")
costLabel.grid(row=14, column=0)

# Inputs
demand = [3,1,2]
draw_beams(demand, 0)
storageCosts = [None, 5, 11, 15]
beamCosts = [None, 4, 7, 8]

canvas.create_text(295, 190, text="Length: 3 feet\nStorage Cost: $5.00\nBeam Cost: $4.00", font="Ariel 20")
canvas.create_text(600, 190, text="Length: 4 feet\nStorage Cost: $11.00\nBeam Cost: $7.00", font="Ariel 20")
canvas.create_text(1050, 190, text="Length: 5 feet\nStorage Cost: $15.00\nBeam Cost: $8.00", font="Ariel 20")


# Feasible Solutions
inventory = [0,0,6]
draw_beams(inventory, 450)

costs=[None,0,0,0]

# Graph
# Nodes
canvas.create_oval(15, 260, 55,300, fill="yellow")
canvas.create_text(35, 280, text="Start")
for i in range(1,4):
    canvas.create_oval(320*i + 15, 260, 320*i + 55,300, fill="orange")
    canvas.create_text(320*i +35, 280, text=(str)(i+2))
    
# Arcs
arcs = np.zeros((4,4))
for i in range(4):
    for j in range(1,4):
        if i < j:
            if j-i == 1:
                canvas.create_line(320*i+55, 280, 320*j+15,280)
                arcs[i,j] = storageCosts[j] + beamCosts[j] * demand[j-1]
                canvas.create_text(160*(i+j) +20, 260, text=(str)(storageCosts[j]) + " + " + (str)(beamCosts[j]) + "(" + (str)(demand[j-1]) + ") = " + (str)(arcs[i,j]), font="Ariel 15")
            elif j-i == 2:
                canvas.create_arc(320*i + 35, 230, 320*j +35, 370, start=180, extent=180, style="arc")
                arcs[i,j] = storageCosts[j] + beamCosts[j] * (demand[j-1] + demand[j-2])
                canvas.create_text(160*(i+j) +20, 350, text=(str)(storageCosts[j]) + " + " + (str)(beamCosts[j]) + "(" + (str)(demand[j-1] + demand[j-2]) + ") = " + (str)(arcs[i,j]), font="Ariel 15")
            else:
                canvas.create_arc(320*i + 35, 170, 320*j +35, 430, start=180, extent=180, style="arc")
                arcs[i,j] = storageCosts[j] + beamCosts[j] * (demand[j-1] + demand[j-2] + demand[j-3])
                canvas.create_text(160*(i+j) +20, 410, text=(str)(storageCosts[j]) + " + " + (str)(beamCosts[j]) + "(" + (str)(demand[j-1] + demand[j-2] + demand[j-3]) + ") = " + (str)(arcs[i,j]), font="Ariel 15")
    
# Costs
canvas.create_text(310, 720, text="+", font="Ariel 20")
canvas.create_text(710, 720, text="+", font="Ariel 20")
canvas.create_text(1000, 720, text="=", font="Ariel 20")

inUse = [False,False,False]
switch(2)

# mainloop
windowOpen=True
while windowOpen:
    try:
        master.update_idletasks()
        master.update()
    except TclError:
        windowOpen=False
        print("window successfully closed")

Now that you have solved a simple example of the inventory problem, try using the following model to solve larger inputs with a graphical visualization.

In [ ]:
# GUI
master = Tk()
master.geometry("1600x750")
master.title("The Inventory Problem")

# Button Functions
started = False
def shuffle():
    global demand
    global started
    started = True
    canvas.delete("pipe")
    canvas.delete("arc")
    canvas.delete("selected")
    canvas.delete("shuffle")
    global on
    on = [False] * 11
    drawOutcome(False)
    # resets demand and inventory to 0 for all sizes
    demand= np.zeros(11)
    inventory= np.zeros(11)
    
    update()
    
    newDemand = [None]* 10
    # last storage and per cost, so that they are always increasing in cost as size increases
    lastS = 1
    lastP = 1
    
    storageLabels = [None] * 10
    perCostLabels = [None] * 10
    # Randomize new storage and pipe costs
    for i in range(10):
        newS = randint(lastS, 40 * (i+1))
        newP = randint(lastP, 10 * (i+1))
        storageCost[i+1] = newS
        perCost[i+1] = newP
        lastS = newS + 1
        lastP = newP + 1
        canvas.create_text(117*(i+1) -24, 500, text="$" + (str)(storageCost[i+1]), font="Courier 20", tags="shuffle")
        canvas.create_text(117*(i+1) -24, 570, text="$" + (str)(perCost[i+1]), font="Courier 20", tags="shuffle")
    # Randomize new demand
    for i in range(10):
        newDemand[i] = randint(1, 10)
        demand[newDemand[i]] +=1
        if demand[newDemand[i]] == 1:
            canvas.create_rectangle(117 * newDemand[i] - 49, 220- 20*newDemand[i], 117*newDemand[i] + 1, 220, outline="#fb0", fill="#fb0", tags="pipe")
            sizeStr= "Size: " + (str)(newDemand[i])
            canvas.create_text(117 * newDemand[i] - 24, 220, fill="darkblue", font="Ariel 10",text=sizeStr, tags="pipe")
            numStr= "Demand: " + (str)((int)(demand[newDemand[i]]))
            canvas.create_text(117 * newDemand[i] - 24, 220- 20*newDemand[i], fill="darkblue", font="Ariel 10",text=numStr, tags=("pipe", "d" + (str)(newDemand[i])))
        else:
            canvas.delete("d" + (str)(newDemand[i]))
            numStr= "Demand: " + (str)((int)(demand[newDemand[i]]))
            canvas.create_text(117 * newDemand[i] - 24, 220- 20*newDemand[i], fill="darkblue", font="Ariel 10",text=numStr, tags=("pipe", "d" + (str)(newDemand[i])))
            
    obj_func()
    k = 10
    loop = True
    global minNode
    while k > 1 and loop:
        if demand[k] > 0:
            minNode = k
            loop = False
        k -=1

# Arcs
A = np.zeros((11,11))
isArc = [[False for j in range(11)] for i in range(11)]
minNode = 10

# Objective Function
def obj_func(): # assigns values to array based on how many characters are in a line from the row's word to column's word and compares this with L
    global A
    global isArc
    isArc = [[False for j in range(11)] for i in range(11)]
    A = np.zeros((11,11))
    for i in range(len(A)):
        for j in range(len(A)): # uses 2D array to iterate over all n words
            if i < j:
                isArc[i][j] = True
                A[i,j] += storageCost[j]
                for k in range(i+1, j+1):  
                    A[i,j] += perCost[j] * demand[k]
        
# dijkstras algorithm
def dijkstras(A, s=0):
    d = [float('inf')] * len(A) # distance of path (cost to get larger pipe)
    p = [float('nan')] * len(A) # previous
    S, F = [], [] # S-searched, F-frontier
    F.append(s)
    d[s] = 0

    while len(F) > 0: # While there are items within the frontier
        F.sort(reverse=True, key=lambda x: d[x])
        f = F.pop() # Takes last item of F as f
        S.append(f) # Settles f
        for w in range(len(A)): # Iterates through the length of A (The amount of pipe lengths)
            if isArc[f][w]: # Check if we are looking at an arc
                if w not in S and w not in F: # If the node is not settled or in the frontier:
                    d[w] = d[f] + A[f][w] # Adds the node's cost to the list of distances
                    p[w] = f # Adds the current node f as the prev for the node being looked at
                    F.append(w) # Add the node being looked at to the frontier.
                else: # If the node being looked at is already in the frontier:
                    if d[f] + A[f][w] < d[w]: # If the path to the node through the current node f is shorter than its previous path:
                        d[w] = d[f] + A[f][w] # Update the distance to the node to be that of the path through F
                        p[w] = f # Update the prev of the node to be f
    return p        
    
# return string output
def formatOutput(p):
    global minNode
    path=[minNode]
    nextNode = minNode
    while not math.isnan(p[nextNode]):
        nextNode = p[nextNode]
        path.append(nextNode)
    return path       
        
        
def optimize():
    canvas.delete("selected")
    canvas.delete("arc")
    global on
    on = [False] * 11
    global inventory
    inventory= np.zeros(11)
    global costVar
    costVar.set("$0")
    nodes = formatOutput(dijkstras(A,s=0))
    for i in nodes:
        if i != 0:
            switch(i)

# Canvas
canvas = Canvas(master, width=1200, height=600, background='white')
canvas.grid(row=0, column=1, rowspan=9, columnspan=11)

# Top Labels
label1 = Label(master, text= "Demand: ", font = "Ariel 20", justify = "left")
label1.grid(row=0, column=0)

label2 = Label(master, text= "Graph: ", font = "Ariel 20", justify = "left")
label2.grid(row=3, column=0)

# Top Buttons
shuffle = Button(master, text="Start / Shuffle", command=shuffle)
shuffle.grid(row=1, column=0)

optimize = Button(master, text="Optimize", command=optimize)
optimize.grid(row=4, column=0)

# Variables
storageCost = [None] * 11
perCost = [None] * 11
totalCost = 0

demand= np.zeros(11) #demand[3] = 5 means there is are 5 demands for a 3 foot beam
inventory= np.zeros(11)

# Cost Labels
storageCostLabel = Label(master, text= "Storage Area Cost: ", font = "Ariel 20", justify = "left")
storageCostLabel.grid(row=7, column=0)

perCostLabel = Label(master, text= "Cost per Beam: ", font = "Ariel 20", justify = "left")
perCostLabel.grid(row=8, column=0)

costLabel = Label(master, text= "Total Cost: ", font = "Ariel 20", justify = "left")
costLabel.grid(row=5, column=0)

costVar = StringVar()
costVar.set("$0")
costVarLabel = Label(master, textvariable= costVar, font = "Ariel 30", justify = "left")
costVarLabel.grid(row=6, column=0)
  
    
# Demand Satisfied/ Unsatisfied Check
def drawOutcome(outcome):
    canvas.delete("outcome")
    if not outcome:
        solved = "red"
        solvedtxt = "Demand Not Met"
    else:
        solved = "green"
        solvedtxt = "Demand Satisfied"
    canvas.create_text(150, 30, fill=solved, font="Ariel 20",text=solvedtxt, tags="outcome")
drawOutcome(False)    
    
def check():
    highestD = 0
    highestI = 0
    for i in range(1, 11):
        if demand[i] > 0:
            highestD = i
        if inventory[i] > 0:
            highestI = i
    if highestD > highestI:
        drawOutcome(False)
    else:
        drawOutcome(True)
        
# Nodes
nodes = [None] * 11
canvas.create_oval(5, 275, 55, 325, fill="yellow")
canvas.create_text(30, 300, text="Start")
for i in range(1, 11):
    canvas.create_oval(117*i - 44, 280, 117*i -4,320, fill="orange")
    canvas.create_text(117*i -24, 300, text=(str)(i))
    

# When Node is added / removed            
on = [False] * 11
def switch(size):
    if started:
        global totalCost
        #turn off
        if on[size]:
            inventory[size] = 0
            on[size] = False
            canvas.delete("a" + (str)(size))
        #turn on
        else:
            on[size] = True
            canvas.create_oval(117*size - 49, 275, 117*size + 1, 325, fill='yellow', tags=("selected","a" + (str)(size)))
            canvas.create_text(117*size -24, 300, text=(str)(size), tags=("selected","a" +(str)(size)))
        update()
        check()
        updatePath()
            

def update():
    global totalCost
    global yourCost
    totalCost = 0
    global inventory
    inventory = np.zeros(11)
    search = True
    i = 10
    atNode = 10
    while search and i >= 1 and atNode>=1:
        loop = True
        atNode = i
        while on[atNode] and i >= 1 and loop:
            inventory[atNode] += demand[i]
            if on[i -1]:
                loop = False
            i -=1
        if inventory[atNode] >= 1:
            totalCost += (int)(storageCost[atNode] + inventory[atNode] * perCost[atNode])
        atNode -=1
        if loop:
            i -=1
    costVar.set("$" + (str)(totalCost))
        
        
def updatePath():
    canvas.delete("arc")
    prev = 0
    for i in range(11):
        if on[i]:
            createArc(prev, i)
            prev = i
            
def createArc(prev, node):
    arcLocation = np.zeros(4)
    textLocation = np.zeros(2)
    if prev == 0:
        arcLocation = [30, 250, 117*node -24, 400]
        textLocation = [58.5*node + 3, 410]
    else:
        arcLocation= [117*prev -24, 250, 117*node -24, 400]
        textLocation = [58.5*(node + prev) - 24, 410]
    canvas.create_arc(arcLocation, start=180, extent=180, style="arc", tags="arc")
    if inventory[node] == 0:
        canvas.create_text(textLocation, text="0", font = "Ariel 12", tags="arc")
    else:
        canvas.create_text(textLocation, text=(str)(storageCost[node]) + "+(" + (str)((int)(inventory[node])) + ")" + (str)(perCost[node]) + "=" + (str)((int)(storageCost[node] + inventory[node] * perCost[node])), font = "Courier 12", tags="arc")
        canvas.create_rectangle(117*node -49, 480, 117*node +1, 590, tags="arc")
    
# Bottom Labels
lengthLabel = Label(master, text= "Length: ", font = "Ariel 20", justify = "left")
lengthLabel.grid(row=10, column=0)
numLabels = [None] * 10
switches = [None] * 10
for i in range(10):
    numLabels[i] = Label(master, text=i+1, font = "Ariel 20", justify = "left")
    numLabels[i].grid(row=9, column=i+2)
    switches[i] = Button(master, text="On / Off", command=partial(switch,i+1))
    switches[i].grid(row=10, column=i+2)
    
# mainloop
windowOpen=True
while windowOpen:
    try:
        master.update_idletasks()
        master.update()
    except TclError:
        windowOpen=False
        print("window successfully closed")